In [ ]:
import pandas as pd
import numpy as np
import datetime
import ipywidgets as widgets
from ipywidgets import (
    IntText,
    FloatText,
    RadioButtons,
    BoundedIntText,
    Checkbox,
    Layout,
    Output,
    Button,
    DatePicker,
    Dropdown,
    Label,
    HBox,
    HTML,
    VBox
)

path = "./weight_loss_dfs/jordan_df.pqt"

In [ ]:
import os

class ParquetCachedDF:
    def __init__(self, file_path):
        self.file_path = file_path
        if os.path.isfile(file_path):
            self.df = pd.read_parquet(file_path)
        else:
            self.df = None
    
    def get_df(self):
        return self.df
    
    def save_df(self):
        if self.df is None:
            raise ValueError("Cannot save empty dataframe")
        else:
            self.df.to_parquet(self.file_path)
            
    def set_df(self, df):
        self.df = df

In [ ]:
class IntInput():
    def __init__(self, label, value=0, step=1, disabled=False, layout=None):
        self.label = HTML(f"<b>{label}:</b>", layout=Layout(width="100px"))
        if isinstance(value, float):
            value = 0
        self.input = IntText(
            value=value,
            step=step,
            disabled=disabled
        )
        
        if layout is None:
            self.layout = Layout(width="max-content")
        else:
            self.layout = layout
        
    def to_box(self):
        return HBox([self.label, self.input], layout=self.layout)
    
    @property
    def value(self):
        return self.input.value
    
    @value.setter
    def value(self, value):
        if not (value is None or np.isnan(value)):
            self.input.value = value
        else:
            self.input.value = 0
    
class FloatInput(IntInput):
    def __init__(self, label, value=0, disabled=False, layout=None):
        super().__init__(label=label, value=0, disabled=disabled, layout=layout)
        self.input = FloatText(
            value=value,
            disabled=disabled
        )
        
    @property
    def value(self):
        return self.input.value
    
    @value.setter
    def value(self, value):
        self.input.value = value
        
class TimeInput():
    def __init__(self, label, value=0, disabled=False, layout=None):
        if layout is None:
            self.layout = Layout(width="max-content")
        else:
            self.layout = layout
            
        self.label = HTML(f"<b>{label}:</b>", layout=Layout(width="100px"))
        
        self._disabled = disabled
        
        self.minutes_input = IntText(
            disabled=self._disabled
        )
        
        self.seconds_input = IntText(
            disabled=self._disabled
        )
        
        self.value = value
        
    @property
    def disabled(self):
        return self._disabled
    
    @disabled.setter
    def disabled(self, value):
        self.minutes_input.disabled = value
        self.seconds_input.disabled = value
        self._disabled = value
    
    @property
    def value(self):
        return self.minutes_input.value + (self.seconds_input.value / 60)
    
    @value.setter
    def value(self, value):
        if isinstance(value, float) and not np.isnan(value):
            self.minutes_input.value = int(value)
            self.seconds_input.value = int((value - int(value)) * 60)
        else:
            self.minutes_input.value = 0
            self.seconds_input.value = 0
    
    def to_box(self):
        return HBox([self.label, self.minutes_input, HTML("m"), self.seconds_input, HTML("s")], layout=self.layout)
    
    
class InchesInput():
    def __init__(self, label, disabled=False, layout=None, value=None):
        self.label = HTML(f"<b>{label}:</b>", layout=Layout(width="100px"))
        self._disabled=disabled
        
        self.inches_input = IntText(
            step=1,
            disabled=self._disabled
        )
        
        self.numerator_input = IntText(
            step=1,
            disabled=self._disabled
        )
        
        self.denominator_input = Dropdown(
            options=[2, 4, 8, 16, 32, 64],
            disabled=self._disabled
        )
        
        self.value = value
        
        if layout is None:
            self.layout = Layout(
                width="max-content"
            )
        else:
            self.layout = layout
            
    def to_box(self):
        return HBox([self.label, self.inches_input, HTML("and"), self.numerator_input, HTML("/"), self.denominator_input])
    
    @property
    def value(self):
        return self.inches_input.value + (self.numerator_input.value / self.denominator_input.value)
    
    @value.setter
    def value(self, value):
        if value is not None and not np.isnan(value):
            value = float(value).as_integer_ratio()
            inches = value[0] // value[1]
            numerator = value[0] % value[1]
            denominator = value[1]
        else:
            inches = 0
            numerator = 0
            denominator = 8
        
        self.inches_input.value = inches
        self.numerator_input.value = numerator
        self.denominator_input.value = denominator if denominator != 1 else 8
    
    @property
    def disabled(self):
        return self.disabled
    
    @disabled.setter
    def disabled(self, value):
        self._disabled = value
        self.inches_input.disabled = value
        self.numerator_input.disabled = value
        self.denominator_input.disabled = value

In [ ]:
def add_row(e):
    global out, path, date, weight, waist, belly, hips, bicep, chest, thigh, target_cals, calories, cutting_bulking, workout, meditate, cardio
    
    idx = str(date.children[1].value)
    if idx == "None":
        out.clear_output()
        display(form)
        display(HTML("Must enter a date!"))
    else:

        row = {
            "Weight": weight.value,
            "Waist": waist.value,
            "Belly": belly.value,
            "Hips": hips.value,
            "Thigh": thigh.value,
            "Bicep": bicep.value,
            "Chest": chest.value,
            "Mile Time": mile_time.value,
            "Target Calories": target_cals.value,
            "Net Calories": calories.value,
            "Mode": mode.children[1].value,
            "Workout": workout.children[1].value,
            "Cardio": cardio.children[1].value,
            "Meditate": meditate.children[1].value,
            "Body Fat": body_fat_percent.value,
            "Resting Heart Rate": heart_rate.value,
        }

        for key in row.keys():
            if not isinstance(row[key], bool):
                if row[key] == 0:
                    row[key] = np.nan

        df_cache = ParquetCachedDF(file_path=path)

        df = df_cache.df

        if df is None:
            df = pd.DataFrame(
                data=row,
                index=[idx]
            )
            df_cache.df = df.sort_index(ascending=False)
            df_cache.save_df()
            with out:
                out.clear_output()
                display(form)
                display(HTML("Success! New dataset created"))
                display(df_cache.df)
        else:
            if idx in df.index:

                def overwrite(e):
                    for k, v in row.items():
                        df.loc[idx, k] = v
                    df_cache.df = df.sort_index(ascending=False)
                    df_cache.save_df()
                    with out:
                        out.clear_output()
                        display(form)
                        display(HTML("Success! Row overwritten with new values"))
                        display(df_cache.df)

                overwrite_btn = Button(
                    description=HTML("Data for date already exists<br>Overwrite row?"),
                    disabled=False,
                    button_style="danger",
                    tooltip="overwrite row",
                    icon="check",
                    layout=Layout(
                        width="max-content"
                    )
                )

                overwrite_btn.on_click(overwrite)

                with out:
                    display(overwrite_btn)

            else:
                df_cache.df = df.append(pd.Series(row, name=idx)).sort_index(ascending=False)
                df_cache.save_df()

                with out:
                    out.clear_output()
                    display(form)
                    display(HTML("Success!!"))
                    display(df_cache.df)



In [ ]:
def set_defaults(date):
    df = ParquetCachedDF(file_path=path).df
    if df is not None and str(date) in df.index:
        df_row = df.loc[str(date)]
        default_weight = df_row["Weight"]
        default_fat = df_row["Body Fat"]
        default_waist = df_row["Waist"]
        default_belly = df_row["Belly"]
        default_hips = df_row["Hips"]
        default_bicep = df_row["Bicep"]
        default_chest = df_row["Chest"]
        default_thigh = df_row["Thigh"]
        default_target_cals = df_row["Target Calories"]
        default_net_cals = df_row["Net Calories"]
        default_hr = df_row["Resting Heart Rate"]
        default_mile_time = df_row["Mile Time"]
        default_cardio = bool(df_row["Cardio"])
        default_workout = bool(df_row["Workout"])
        default_meditate = bool(df_row["Meditate"])
        default_mode = df_row["Mode"]
    else:
        default_weight = 0
        default_fat = 0
        default_waist = 0
        default_belly = 0
        default_hips = 0
        default_bicep = 0
        default_chest = 0
        default_thigh = 0
        default_target_cals = 0 
        default_net_cals = 0
        default_hr = 0
        default_mile_time = 0
        default_cardio = False
        default_workout = False
        default_meditate = False
        default_mode = "Cutting"
        
        # woefully, must do this string-date-datetime conversion to get dates to a format that can be subtracted
        datetime_selected = datetime.datetime(day=date.day, month=date.month, year=date.year)
        datetime_last_entry = datetime.datetime.strptime(df.iloc[0].name, "%Y-%m-%d")
        days_since_last_entry = (datetime_selected - datetime_last_entry).days
        if days_since_last_entry == 1:
            default_target_cals = df.iloc[0]["Target Calories"]
            
    weight.value = default_weight
    body_fat_percent.value = default_fat
    waist.value = default_waist
    belly.value = default_belly
    hips.value = default_hips
    bicep.value = default_bicep
    chest.value = default_chest
    thigh.value = default_thigh
    target_cals.value = default_target_cals
    calories.value = default_net_cals
    heart_rate.value = default_hr
    mile_time.value = default_mile_time
    cardio.children[1].value = default_cardio
    workout.children[1].value = default_workout
    meditate.children[1].value = default_meditate
    mode.children[1].value = default_mode

In [ ]:
# initial setup
df = ParquetCachedDF(file_path=path).df
out = Output()
current_date = datetime.date.today()
layout=Layout(
    width="max-content"
)

# date change handler
def on_date_entry_event(change):
    df = ParquetCachedDF(file_path=path).df
    new_date = change["new"]
    set_defaults(date=new_date)

# widgets
date_picker = DatePicker(
    disabled=False,
    value=current_date
)
date_picker.observe(on_date_entry_event, names="value")
date = HBox(
    [HTML("<b>Date</b>:", layout=Layout(width="100px")), date_picker]
)
weight = FloatInput(label="Weight")
body_fat_percent = FloatInput(label="Body Fat %")
waist = InchesInput(label="Waist")
belly = InchesInput(label="Belly")
hips = InchesInput(label="Hips")
bicep = InchesInput(label="Bicep")
chest = InchesInput(label="Chest")
thigh = InchesInput(label="Thigh")
target_cals = IntInput(label="Target Calories")
calories = IntInput(label="Net Calories")
heart_rate = IntInput(label="Resting HR")
mile_time = TimeInput(label="Mile Time")
mode = HBox(
    [
        HTML("<b>Mode</b>:"),
        RadioButtons(
            options=["Cutting", "Bulking", "Deload"],
        )
    ]
)
workout = HBox(
    [
        HTML("<b>Workout</b>:"),
        Checkbox(
            disabled=False,
            indent=False,
        )
    ]
)
cardio = HBox(
    [
        HTML("<b>Cardio</b>:"),
        Checkbox(
            disabled=False,
            indent=False,
        )
    ]
)

meditate = HBox(
    [
        HTML("<b>Meditate</b>:"),
        Checkbox(
            disabled=False,
            indent=False,
        )
    ]
)
submit = Button(
    description="Submit",
    disabled=False,
    button_style="success",
    tooltip="submit form",
    icon="check",
    layout=Layout(
        width="max-content"
    )
)

submit.on_click(add_row)

form = VBox(
    [
        date,
        weight.to_box(),
        waist.to_box(),
        belly.to_box(),
        hips.to_box(),
        thigh.to_box(),
        bicep.to_box(),
        chest.to_box(),
        heart_rate.to_box(),
        mile_time.to_box(),
        target_cals.to_box(),
        calories.to_box(),
        body_fat_percent.to_box(),
        mode,
        workout,
        cardio,
        meditate,
        submit
    ],
    layout=Layout(
        width="100%"
    )
)

display(out)
set_defaults(date=current_date)
with out:
    display(form)
    display(df.head())